In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [3]:
# Data Augmentation for Training
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
    "Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold2_AUG/Train/",
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' for one-hot encoded labels
)

Found 7126 images belonging to 6 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
validation_set = test_datagen.flow_from_directory(
    "Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold2_AUG/Train/",
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',  # Use 'categorical' for one-hot encoded labels
)

Found 7126 images belonging to 6 classes.


In [5]:
# Custom F1 Score Metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Update the precision and recall for each batch
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        # Calculate F1 score as the harmonic mean of precision and recall
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [6]:
# Build the Bi-GRU Model
bi_gru_model = tf.keras.models.Sequential()

In [7]:
# Flatten and Reshape for Bi-GRU
bi_gru_model.add(tf.keras.layers.Flatten(input_shape=(64, 64, 3)))
bi_gru_model.add(tf.keras.layers.Reshape((16, -1)))  # Reshape to (time_steps, features)

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
# Bidirectional GRU Layer
bi_gru_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=64, activation='tanh', return_sequences=False)))

In [9]:
# Fully Connected Layers
bi_gru_model.add(tf.keras.layers.Dense(units=128, activation='relu'))
bi_gru_model.add(tf.keras.layers.Dense(6, activation='softmax'))  # 6 classes

In [10]:
# Compile the model with Precision, Recall, and F1 Score
bi_gru_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Use categorical_crossentropy for one-hot encoded labels
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        F1Score(name='f1_score')  # Add custom F1 score metric
    ]
)

In [11]:
# Train the model
history = bi_gru_model.fit(
    x=training_set,
    validation_data=validation_set,
    epochs=100
)

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 656ms/step - accuracy: 0.3337 - f1_score: 0.0901 - loss: 1.6706 - precision: 0.4109 - recall: 0.0508

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


223/223 ━━━━━━━━━━━━━━━━━━━━ 242s 979ms/step - accuracy: 0.3337 - f1_score: 0.0902 - loss: 1.6705 - precision: 0.4112 - recall: 0.0509 - val_accuracy: 0.3649 - val_f1_score: 0.1953 - val_loss: 1.5946 - val_precision: 0.5560 - val_recall: 0.1184
Epoch 2/100
223/223 ━━━━━━━━━━━━━━━━━━━━ 202s 906ms/step - accuracy: 0.3569 - f1_score: 0.1128 - loss: 1.5928 - precision: 0.6242 - recall: 0.0622 - val_accuracy: 0.3741 - val_f1_score: 0.0995 - val_loss: 1.5556 - val_precision: 0.7379 - val_recall: 0.0533
Epoch 3/100
223/223 ━━━━━━━━━━━━━━━━━━━━ 203s 910ms/step - accuracy: 0.3805 - f1_score: 0.1550 - loss: 1.5327 - precision: 0.6400 - recall: 0.0882 - val_accuracy: 0.4178 - val_f1_score: 0.1929 - val_loss: 1.4726 - val_precision: 0.6373 - val_recall: 0.1137
Epoch 4/100
223/223 ━━━━━━━━━━━━━━━━━━━━ 219s 984ms/step - accuracy: 0.4078 - f1_score: 0.1827 - loss: 1.4948 - precision: 0.6178 - recall: 0.1074 - val_accuracy: 0.4171 - val_f1_score: 0.2279 - val_loss: 1.4407 - val_precision: 0.6649 - val

In [12]:
# Save Loss plot
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('loss_plot_bigru.png')  # Save the plot as a PNG file
plt.close()

# Save Accuracy plot
plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('accuracy_plot_bigru.png')  # Save the plot as a PNG file
plt.close()

# Save Precision plot
plt.figure()
plt.plot(history.history['precision'], label='Train Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')
plt.legend()
plt.title("Precision")
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.savefig('precision_plot_bigru.png')  # Save the plot as a PNG file
plt.close()

# Save Recall plot
plt.figure()
plt.plot(history.history['recall'], label='Train Recall')
plt.plot(history.history['val_recall'], label='Validation Recall')
plt.legend()
plt.title("Recall")
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.savefig('recall_plot_bigru.png')  # Save the plot as a PNG file
plt.close()

# Save F1 Score plot
plt.figure()
plt.plot(history.history['f1_score'], label='Train F1 Score')
plt.plot(history.history['val_f1_score'], label='Validation F1 Score')
plt.legend()
plt.title("F1 Score")
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.savefig('f1_score_plot_bigru.png')  # Save the plot as a PNG file
plt.close()

In [13]:
bi_gru_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 12288)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 16, 768)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │         320,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,012,628 (3.86 MB)

 Trainable params: 337,542 (1.29 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 675,086 (2.58 MB)